In [47]:
aiken check

    Compiling aiken-lang/stdlib main (/config/workspace/repo/build/packages/aiken-lang-stdlib)
    Compiling aiken-lang/hello_world 0.0.0 (/config/workspace/repo)

Summary
    0 errors, 0 warnings


In [48]:
aiken build

jq . plutus.json

    Compiling aiken-lang/stdlib main (/config/workspace/repo/build/packages/aiken-lang-stdlib)
    Compiling aiken-lang/hello_world 0.0.0 (/config/workspace/repo)
   Generating project's blueprint (/config/workspace/repo/plutus.json)

Summary
    0 errors, 0 warnings
{
  "preamble": {
    "title": "aiken-lang/hello_world",
    "description": "Aiken contracts for project 'aiken-lang/hello_world'",
    "version": "0.0.0",
    "plutusVersion": "v2",
    "license": "Apache-2.0"
  },
  "validators": [
    {
      "title": "hello_world.hello_world",
      "datum": {
        "title": "datum",
        "schema": {
          "$ref": "#/definitions/hello_world~1Datum"
        }
      },
      "redeemer": {
        "title": "redeemer",
        "schema": {
          "$ref": "#/definitions/hello_world~1Redeemer"
        }
      },
      "compiledCode": "5901f30100003232323232323232323232322223232533300a3232533300c002100114a06464660026eb0cc024c02ccc024c02c019200048040dd7198049805802240006002002444a66

In [49]:
SCRIPT_ADDRESS=$(aiken address)
echo $SCRIPT_ADDRESS


Summary
    0 errors, 0 warnings
addr_test1wpjn04938grlh627nmvu3xepy2pessy76pdrae6jz765dqs9dfswe


In [50]:
if [ ! -e "wallet.sk" ]; then
    cardano-cli address key-gen --verification-key-file wallet.vk --signing-key-file wallet.sk
fi


In [52]:
WALLET_ADDRESS=$(cardano-cli address build \
    --payment-verification-key-file ./wallet.vk \
    --testnet-magic $CARDANO_NODE_MAGIC)

echo "my wallet address: $WALLET_ADDRESS"

WALLET_KEYHASH=$(cardano-cli address key-hash \
    --payment-verification-key-file wallet.vk)

echo "my wallet keyhash: $WALLET_KEYHASH"


my wallet address: addr_test1vrg876takh3nhv8k4u8u3372x8r48fnjmj9zcd9cctunqxces80wm
my wallet keyhash: d07f697db5e33bb0f6af0fc8c7ca31c753a672dc8a2c34b8c2f9301b


In [53]:
cardano-cli query utxo \
    --address $WALLET_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --out-file wallet.state

jq . wallet.state

{
  "777462ef575c6a4efc422a598b00146bd7792169a13f94608c0a723fef62d236#1": {
    "address": "addr_test1vrg876takh3nhv8k4u8u3372x8r48fnjmj9zcd9cctunqxces80wm",
    "datum": null,
    "datumhash": null,
    "inlineDatum": null,
    "referenceScript": null,
    "value": {
      "lovelace": 189832475
    }
  }
}


In [54]:
INPUT_UTXO=$(jq -r 'keys[0]' wallet.state)

echo $INPUT_UTXO

777462ef575c6a4efc422a598b00146bd7792169a13f94608c0a723fef62d236#1


In [55]:
echo "{\"constructor\" : 0, \"fields\": [{ \"bytes\": \"$WALLET_KEYHASH\" }]}" > datum.json

jq . datum.json

{
  "constructor": 0,
  "fields": [
    {
      "bytes": "d07f697db5e33bb0f6af0fc8c7ca31c753a672dc8a2c34b8c2f9301b"
    }
  ]
}


In [56]:
cardano-cli transaction build \
    --babbage-era \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --tx-in $INPUT_UTXO \
    --tx-out $SCRIPT_ADDRESS+10000000 \
    --tx-out-inline-datum-file datum.json \
    --change-address $WALLET_ADDRESS \
    --out-file lock.tx

cardano-cli transaction view --tx-file lock.tx

Estimated transaction fee: Lovelace 167525
auxiliary scripts: null
certificates: null
collateral inputs: []
era: Babbage
fee: 167525 Lovelace
inputs:
- 777462ef575c6a4efc422a598b00146bd7792169a13f94608c0a723fef62d236#1
metadata: null
mint: null
outputs:
- address: addr_test1wpjn04938grlh627nmvu3xepy2pessy76pdrae6jz765dqs9dfswe
  address era: Shelley
  amount:
    lovelace: 10000000
  datum:
    constructor: 0
    fields:
    - bytes: d07f697db5e33bb0f6af0fc8c7ca31c753a672dc8a2c34b8c2f9301b
  network: Testnet
  payment credential script hash: 6537d4b13a07fbe95e9ed9c89b21228398409ed05a3ee75217b54682
  reference script: null
  stake reference: null
- address: addr_test1vrg876takh3nhv8k4u8u3372x8r48fnjmj9zcd9cctunqxces80wm
  address era: Shelley
  amount:
    lovelace: 179664950
  datum: null
  network: Testnet
  payment credential key hash: d07f697db5e33bb0f6af0fc8c7ca31c753a672dc8a2c34b8c2f9301b
  reference script: null
  stake reference: null
reference inputs: []
required signers (payme

In [57]:
cardano-cli transaction sign \
    --tx-file lock.tx \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --signing-key-file wallet.sk \
    --out-file lock.signed

In [58]:
cardano-cli transaction submit --testnet-magic $CARDANO_NODE_MAGIC --tx-file lock.signed

Transaction successfully submitted.


In [73]:
cardano-cli query utxo \
    --address $SCRIPT_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --out-file script.state

jq . script.state

{
  "49cc80a49f61f5e4a543069469141b4d4eef97e7f636ea73e2708b9df8f73c9e#0": {
    "address": "addr_test1wpjn04938grlh627nmvu3xepy2pessy76pdrae6jz765dqs9dfswe",
    "datum": null,
    "inlineDatum": {
      "constructor": 0,
      "fields": [
        {
          "bytes": "d07f697db5e33bb0f6af0fc8c7ca31c753a672dc8a2c34b8c2f9301b"
        }
      ]
    },
    "inlineDatumhash": "2904c8093bc1f1855817f77f3a0d7c16f7499e65a97d087f409a476c664a07c2",
    "referenceScript": null,
    "value": {
      "lovelace": 10000000
    }
  }
}


In [74]:
TO_UNLOCK_UTXO=$(jq -r 'keys[0]' script.state)

echo $TO_UNLOCK_UTXO

49cc80a49f61f5e4a543069469141b4d4eef97e7f636ea73e2708b9df8f73c9e#0


In [65]:
cardano-cli query utxo \
    --address $WALLET_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --out-file wallet.state

jq . wallet.state

{
  "49cc80a49f61f5e4a543069469141b4d4eef97e7f636ea73e2708b9df8f73c9e#1": {
    "address": "addr_test1vrg876takh3nhv8k4u8u3372x8r48fnjmj9zcd9cctunqxces80wm",
    "datum": null,
    "datumhash": null,
    "inlineDatum": null,
    "referenceScript": null,
    "value": {
      "lovelace": 179664950
    }
  }
}


In [75]:
COLLATERAL_UTXO=$(jq -r 'keys[0]' wallet.state)

echo $COLLATERAL_UTXO

49cc80a49f61f5e4a543069469141b4d4eef97e7f636ea73e2708b9df8f73c9e#1


In [63]:
VALUE=$(echo "Hello, World!" | xxd -p )

echo "{\"constructor\" : 0, \"fields\": [{ \"bytes\": \"$VALUE\" }]}" > redeemer.json

jq . redeemer.json

{
  "constructor": 0,
  "fields": [
    {
      "bytes": "48656c6c6f2c20576f726c64210a"
    }
  ]
}


In [70]:
cardano-cli query protocol-parameters --testnet-magic $CARDANO_NODE_MAGIC > params.json

jq . params.json

{
  "collateralPercentage": 150,
  "costModels": {
    "PlutusScriptV1": {
      "addInteger-cpu-arguments-intercept": 205665,
      "addInteger-cpu-arguments-slope": 812,
      "addInteger-memory-arguments-intercept": 1,
      "addInteger-memory-arguments-slope": 1,
      "appendByteString-cpu-arguments-intercept": 1000,
      "appendByteString-cpu-arguments-slope": 571,
      "appendByteString-memory-arguments-intercept": 0,
      "appendByteString-memory-arguments-slope": 1,
      "appendString-cpu-arguments-intercept": 1000,
      "appendString-cpu-arguments-slope": 24177,
      "appendString-memory-arguments-intercept": 4,
      "appendString-memory-arguments-slope": 1,
      "bData-cpu-arguments": 1000,
      "bData-memory-arguments": 32,
      "blake2b_256-cpu-arguments-intercept": 117366,
      "blake2b_256-cpu-arguments-slope": 10475,
      "blake2b_256-memory-arguments": 4,
      "cekApplyCost-exBudgetCPU": 23000,
      "cekApplyCost-exBudgetMemory": 100,
      "cekBuiltinCos

In [72]:
cardano-cli transaction build \
    --babbage-era \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --protocol-params-file params.json \
    --tx-in $TO_UNLOCK_UTXO \
    --tx-in-redeemer-file redeemer.json \
    --tx-in-inline-datum-present \
    --tx-in-script-file script.json \
    --tx-in-collateral $COLLATERAL_UTXO \
    --change-address $WALLET_ADDRESS \
    --out-file unlock.tx

cardano-cli transaction view --tx-file unlock.tx

Command failed: transaction build  Error: The following scripts have execution failures:
the script for transaction input 0 (in the order of the TxIds) failed with: 
The redeemer pointer: RdmrPtr Spend 0 points to a Plutus script that does not exist.
The pointers that can be resolved are: fromList [(RdmrPtr Spend 0,(Spending (TxIn (TxId {_unTxId = SafeHash "49cc80a49f61f5e4a543069469141b4d4eef97e7f636ea73e2708b9df8f73c9e"}) (TxIx 0)),Nothing,ScriptHash "6537d4b13a07fbe95e9ed9c89b21228398409ed05a3ee75217b54682"))]

Command failed: transaction view  Error: unlock.tx: unlock.tx: openFile: does not exist (No such file or directory)


: 1